In [1]:
import psycopg2
import logging
import pandas as pd

In [28]:
logger = logging.getLogger()

conn = None
try:
    conn = psycopg2.connect(host="tableau.lendingworks.co.uk",
                            port=8060,
                            database="workgroup",
                            user="readonly",
                            password="HWWwZGmFCN7ikoN98hirG9kBbBVHpr66yHNfBqAmY6oW9DXkYy")

    curr = conn.cursor()

    query = "\
        SELECT\
            d.server,\
            d.has_extract,\
            w.name,\
            s.name,\
            p.name\
        FROM\
            public.data_connections d\
        JOIN\
            public.workbooks w\
        ON\
            w.id = d.owner_id\
        JOIN\
            public.projects p\
        ON\
            w.project_id = p.id\
        JOIN\
            public.users u\
        ON\
            u.id = w.owner_id\
        JOIN\
            public.system_users s\
        ON\
            S.id = U.system_user_id;"   

    curr.execute(query)
    conn.commit()

    df = pd.DataFrame(curr.fetchall(), columns=[desc[0] for desc in curr.description])


except Exception as error: 
    logger.error(f"Error while connecting to server repositry. {error}")
    logger.error(error, exc_info=True)

finally:
    if conn:  
        curr.close()
        conn.close()

In [37]:
wokrbooks = df[(df["server"]=="jdbc:awsathena://AwsRegion=eu-west-1;")&(df["has_extract"]==True)][["name"]]

wokrbooks.columns.values[0] = "Workbook"
wokrbooks.columns.values[1] = "Workbook Owner"
wokrbooks.columns.values[2] = "Project (folder)"
wokrbooks["Data Source Server"] = df[(df["server"]=="jdbc:awsathena://AwsRegion=eu-west-1;")&(df["has_extract"]==True)][["server"]]

In [42]:
wokrbooks.drop_duplicates().to_csv("Workbooks_with_athena_connection.csv", index = False)
